# Workflow for deepTools ChIP-Seq Heatmap Generation

The following notebook shows the optimized workflow for creating heatmaps starting from wig.gz files. Will use a combination of UCSC and deepTools utilities.

In [3]:
!pip freeze #Python packages and versions used in the conda environment

accessible-pygments==0.0.4
adjustText==0.8
aiohttp==3.9.3
aiohttp-cors==0.7.0
aiosignal==1.3.1
alabaster==0.7.13
anndata==0.9.2
annotated-types==0.6.0
annoy==1.17.3
anyio==4.2.0
appdirs==1.4.4
arboreto==0.1.6
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
async-lru==2.0.4
async-timeout==4.0.3
attr==0.3.2
attrs==23.2.0
Babel==2.14.0
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
backports.zoneinfo==0.2.1
bbknn==1.6.0
beautifulsoup4==4.12.3
bidict==0.22.1
bioservices==1.11.2
bleach==6.1.0
blessed==1.20.0
blosc2==2.0.0
bokeh==3.1.1
boltons==23.1.1
bs4==0.0.2
cachetools==5.3.2
cattrs==23.2.3
certifi==2024.2.2
cffi==1.16.0
charset-normalizer==3.3.2
click==8.1.7
cloudpickle==3.0.0
colorama==0.4.6
colorful==0.5.6
colorlog==6.8.2
comm==0.2.1
contourpy==1.1.1
cryptography==42.0.2
ctxcore==0.2.0
cycler==0.12.1
Cython==0.29.37
cytoolz==0.12.3
dask==2023.5.0
dataclasses-json==0.6.4
debugpy @ file

# Tools download and setup

In [4]:
#Download the python program wigToBigWig from the UCSC's utilties library.

!wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/wigToBigWig
!chmod +x ./wigToBigWig

#If you haven't done so yet, download deepTools using the following line

!pip install deeptools==3.5.4.post1

#When generating BigWig files, will need a file containing chromosome sizes

!wget https://hgdownload.cse.ucsc.edu/goldenpath/mm10/bigZips/mm10.chrom.sizes
chrom_size = "mm10.chrom.sizes"

# Wig Files List Generator

To automate the process, we will generate a list that will find the right file according to a sample_key.txt file and implement it accordingly. In our dataset, the wig files were labeled using an index code that would only be deciphered using the sample_key to figure out histone modification mark and their experimental group.

In [2]:
import pandas as pd

# sample_key.txt (file with name conversion and other info) has been changed to a csv file through excel

key = pd.read_csv("sample_key.csv")
key.head()

,project,assays,species,mark,sample name,index,Unnamed: 6,Unnamed: 7
0,Underhill,Chipseq,mouse,H3K4me1,SSM2_09June23_D_H3K4me1,PX3242_TGACCA-CAGTACAG,NaN,NaN
1,Underhill,Chipseq,mouse,H3K4me3,SSM2_09June23_D_H3K4me3,PX3242_CTTGTA-CAGTACAG,NaN,NaN
2,Underhill,Chipseq,mouse,H3K9me3,SSM2_09June23_D_H3K9me3,PX3242_AAGCGA-CAGTACAG,NaN,NaN
3,Underhill,Chipseq,mouse,H3K27me3,SSM2_09June23_D_H3K27me3,PX3242_ACTCTC-CAGTACAG,NaN,NaN
4,Underhill,Chipseq,mouse,H3K36me3,SSM2_09June23_D_H3K36me3,PX3242_ATACGG-CAGTACAG,NaN,NaN


In [3]:
# Filter the dataframe for only rows that correspond to ChIP-Seq data (marked as Chipseq in the assays column)

ChIP_Seq = key[key["assays"] == "Chipseq"]
ChIP_Seq

,project,assays,species,mark,sample name,index,Unnamed: 6,Unnamed: 7
0,Underhill,Chipseq,mouse,H3K4me1,SSM2_09June23_D_H3K4me1,PX3242_TGACCA-CAGTACAG,NaN,NaN
1,Underhill,Chipseq,mouse,H3K4me3,SSM2_09June23_D_H3K4me3,PX3242_CTTGTA-CAGTACAG,NaN,NaN
2,Underhill,Chipseq,mouse,H3K9me3,SSM2_09June23_D_H3K9me3,PX3242_AAGCGA-CAGTACAG,NaN,NaN
3,Underhill,Chipseq,mouse,H3K27me3,SSM2_09June23_D_H3K27me3,PX3242_ACTCTC-CAGTACAG,NaN,NaN
4,Underhill,Chipseq,mouse,H3K36me3,SSM2_09June23_D_H3K36me3,PX3242_ATACGG-CAGTACAG,NaN,NaN
5,Underhill,Chipseq,mouse,H3K27ac,SSM2_09June23_D_H3K27ac,PX3242_CACGAT-CAGTACAG,NaN,NaN
6,Underhill,Chipseq,mouse,H3K36me2,SSM2_09June23_D_H3K36me2,PX3242_GAGTGG-CAGTACAG,NaN,NaN
7,Underhill,Chipseq,mouse,Ubiquityl,SSM2_09June23_D_Ubiquityl,PX3242_CTGCTG-CAGTACAG,NaN,NaN
8,Underhill,Chipseq,mouse,Input,SSM2_09June23_D_Input,PX3242_TTCTCC-GTAGTACT,NaN,NaN
9,Underhill,Chipseq,mouse,H3K4me1,CTL_Tongue_10Aug23_G_H3K4me1,PX3242_ACAGTG-GTAGTACT,NaN,NaN


In [4]:
# Find all the unique markers used in the ChIP-Seq data to form pairs of CTL vs SSM2 later (found in the mark column)

marks = set(ChIP_Seq["mark"])
marks

{'H3K27ac',
 'H3K27me3',
 'H3K36me2',
 'H3K36me3',
 'H3K4me1',
 'H3K4me3',
 'H3K9me3',
 'Input',
 'Ubiquityl'}

In [5]:
# Make empty lists that will hold the string names of the markers, CTL index names, and the SSM2 index names

mark_list = []
CTL_list = []
SSM2_list = []

In [6]:
# The following is a for loop that will ultimately scan, pull, and insert the index names of CTL and SSM2 bigwig files pairs

for mark in marks:  # A for loop that will go through each unique marker name from the dataframe
    CTL = ChIP_Seq[(ChIP_Seq["mark"] == mark) & (ChIP_Seq["sample name"].str.contains("CTL"))] # Collect row that contains the right marker type and has the substring "CTL" in the sample name column
    CTL_index = CTL["index"].values[0] # Copy just the string of the index name from the CTL row. Have to use .values[0] or else you get a string that contains the row number and other useless data that will interfere with getting the right bigwig file name.
    SSM2 = ChIP_Seq[(ChIP_Seq["mark"] == mark) & (ChIP_Seq["sample name"].str.contains("SSM2"))] # Repeat of the CTL portion, but instead we find the row with the SSM2
    SSM2_index = SSM2["index"].values[0]
    CTL_bw = CTL_index + ".markdup.q5.F3332.PET.bw" # Add the remaining text from the bigwig file in order to call it properly
    SSM2_bw = SSM2_index + ".markdup.q5.F3332.PET.bw"
    mark_list.append(mark) # Add the current marker name to the mark_list list
    CTL_list.append(CTL_bw) # Add the full name of the CTL bigwig file to the CTL_list list
    SSM2_list.append(SSM2_bw) # Add the full name of the SSM2 bigwig file to the SSM2_list list
    

In [6]:
import pyranges as pr
import os

#Prepare bed files containing the start and end positions of genes using the USCS table browser tool. Specifically, used the NCBI dataset. Checked bed file using IGV and comp

cluster_1 = "mm10_cluster1_genelist_NCBI.bed"
cluster_2 = "mm10_cluster2_genelist_NCBI.bed"
cluster_3 = "mm10_cluster3_genelist_NCBI.bed"
Hox_genes = "mm10_HoxGenes_genelist_NCBI.bed"


In [7]:
# Make a dataframe that will house columns for marker type, and CTL & SSM2 bigwig names.

bw_table = pd.DataFrame({'mark':mark_list, 'CTL':CTL_list, 'SSM2':SSM2_list})
bw_table

,mark,CTL,SSM2
0,H3K27me3,PX3242_ACTGAT-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_ACTCTC-CAGTACAG.markdup.q5.F3332.PET.bw
1,Ubiquityl,PX3242_GAAACC-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_CTGCTG-CAGTACAG.markdup.q5.F3332.PET.bw
2,H3K4me1,PX3242_ACAGTG-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_TGACCA-CAGTACAG.markdup.q5.F3332.PET.bw
3,H3K4me3,PX3242_AAACAT-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_CTTGTA-CAGTACAG.markdup.q5.F3332.PET.bw
4,H3K36me2,PX3242_CCGCAA-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_GAGTGG-CAGTACAG.markdup.q5.F3332.PET.bw
5,H3K27ac,PX3242_CACTCA-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_CACGAT-CAGTACAG.markdup.q5.F3332.PET.bw
6,Input,PX3242_GTGAAA-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_TTCTCC-GTAGTACT.markdup.q5.F3332.PET.bw
7,H3K36me3,PX3242_ATCCTA-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_ATACGG-CAGTACAG.markdup.q5.F3332.PET.bw
8,H3K9me3,PX3242_AAGGAC-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_AAGCGA-CAGTACAG.markdup.q5.F3332.PET.bw


# Converting Wig Files to BigWig

BigWig (bw) files are necessary due to deepTools using that format as their input, but also they come in handy when utilzing IGV program. When trying to open wig files in IGV, they will reccomend converting it to TDF using their built-in tools, but that can take hours to process. Instead, converting to bw format takes only a couple of minutes. We will use the program wigToBigWig, however we must first unzip the files if that are compressed as .gz with a linux function called gzip.

In [13]:
# Using gzip to unzip the .gz compress wig files.
# To automate this process, run a loop to gather all the names of the files from the samples_key file, and then feed those names through another loop that will unzip each one sequentially.

compressed_list = []

for row in ChIP_Seq:
    compressed_list.append(ChIP_Seq["index"].values[0] + ".markdup.q5.F3332.PET.wig.gz")
for name in compressed_list:
    !gzip -dv -k {name}

['PX3242_TGACCA-CAGTACAG.markdup.q5.F3332.PET.wig.gz', 'PX3242_TGACCA-CAGTACAG.markdup.q5.F3332.PET.wig.gz', 'PX3242_TGACCA-CAGTACAG.markdup.q5.F3332.PET.wig.gz', 'PX3242_TGACCA-CAGTACAG.markdup.q5.F3332.PET.wig.gz', 'PX3242_TGACCA-CAGTACAG.markdup.q5.F3332.PET.wig.gz', 'PX3242_TGACCA-CAGTACAG.markdup.q5.F3332.PET.wig.gz', 'PX3242_TGACCA-CAGTACAG.markdup.q5.F3332.PET.wig.gz', 'PX3242_TGACCA-CAGTACAG.markdup.q5.F3332.PET.wig.gz']


In [ ]:
#Generating bw files from the wig files using UCSC's wigToBigWig tool

unzipped_list = []
bw_list = []

for row in ChIP_Seq:
    unzipped_list.append(ChIP_Seq["index"].values[0] + ".markdup.q5.F3332.PET.wig")
    bw_list.append(ChIP_Seq["index"].values[0] + + ".markdup.q5.F3332.PET.bw")

for i in range(unzipped_list):
    !./wigToBigWig {unzipped_list[i]} {chrom_sizes} {bw_list[i]} -clip

# Log2 normalizing BigWig files

For some of the heatmaps, we will use log2 normalization, which requires using the deeptools tool bigwigCompare. Two different bw files are needed, treatment bw file and a input bw file.

In [8]:
#The following code is used if log2 normalization is needed for the heatmap.
#Isolate the input row which will be used as the reference bw file during log2 normalization.

inpute = bw_table[bw_table.eq("Input").any(axis=1)]

#Make a for loop that will run through the bw_table list and utilize deeptools' bigWigCompare to normalize every bw file to its input file (CTL or SSM2 respectively) with bin sizes of 25.
#Takes ~40 mins to run log2 normalization from ~1 GB bw files. Normalized bw files are around 600 MB. Number of processors is set to 20 - change accordingly.

for i in range(len(bw_table['mark'])):
    if bw_table.loc[i, 'mark'] != "Input":
        !bigwigCompare -b1 {bw_table.loc[i, 'CTL']} -b2 {result['CTL'].values[0]} -bs 25 --operation log2 -o {bw_table.loc[i,'mark'] + "_CTL_log2.bw"} -p 20
        !bigwigCompare -b1 {bw_table.loc[i, 'SSM2']} -b2 {result['SSM2'].values[0]} -bs 25 --operation log2 -o {bw_table.loc[i,'mark'] + "_SSM2_log2.bw"} -p 20

# Generating Heatmaps

Before creating a heatmap, we must first generate an intermediate matrix file that will house score values of different genes upstream and downstream of the region of interest based on the bw file and a reference region bed file (gene list).

We used a variety of bed file that either included all the genes found in the mm10 genomes or a curated gene list.
1. For the former, it can be downloaded from [here](https://sourceforge.net/projects/rseqc/files/BED/Mouse_Mus_musculus/mm10_UCSC_knownGene.bed.gz/download).
2. For the latter, it can be generated using [UCSC's table browser website](https://genome.ucsc.edu/cgi-bin/hgTables?hgsid=2125137146_54BnotAR6qPQ2C8qFACtCzdewo7Y&clade=mammal&org=Mouse&db=mm10&hgta_group=genes&hgta_track=refSeqComposite&hgta_table=refGene&hgta_regionType=genome&position=chr12%3A56%2C694%2C976-56%2C714%2C605&hgta_outputType=primaryTable&hgta_outFileName=). From trial runs, the most polished bed file was generated using the NCBI refseq track and UCSC refseq table. For identifiers, input a txt file that contains a list of genes. Export as a bed file with no headers. Due to the gene data set, they will occassionally have multiple versions of the same gene list, creating repetitions of one single gene in the bed file. This will mess up the heatmap due to the pressence of large and broad streaks. To fix this, open the bed file using a txt file editor (for example: Notepad on Windows) and delete rows that have similar chromosome, start, and end values.

# Heatmaps around TSS, using absolute values, and all mm10 genes

The following will generate pairs of heatmaps for each histone modification with the CTL group on the left and SSM2 group on the right

In [ ]:
# Using the bw_table, we will now go through each row/marker type and input the CTL and SSM2 bigwig files into the deeptools computeMatrix program
# This will create a matrix from both CTL and SSM2 bigwigs that can later be used to create heatmaps

for i in range(len(bw_table['mark'])):
    !computeMatrix reference-point -S {bw_table.loc[i, 'CTL']} {bw_table.loc[i, 'SSM2']} -R mm10_UCSC_knownGene.bed -bs 25 -b 5000 -a 5000  -out {bw_table.loc[i, 'mark'] + "_CTL_SSM2_referencePoint.matrix"} --missingDataAsZero --skipZeros --samplesLabel {bw_table.loc[i, 'mark'] + "_CTL"} {bw_table.loc[i, 'mark'] + "_SSM2"} -p 20

# Similar to the code above for running computeMatrix, we will use the bw_table to run deeptool's plotHeatmap
# This will make heatmap pairs of CTL and SSM2

for i in range(len(bw_table['mark'])):
    !plotHeatmap -m {bw_table.loc[i, 'mark'] + "_CTL_SSM2_referencePoint.matrix"} --colorMap RdBu --heatmapHeight 25 --heatmapWidth 5 -out {bw_table.loc[i, 'mark'] + "_CTL_SSM2_referencePoint.svg"} --sortUsing max --regionsLabel genes --legendLocation none

# Heatmaps around TSS, using log2 normalized values, and all mm10 genes

The following will generate pairs of heatmaps for each histone modification with the CTL group on the left and SSM2 group on the right

In [ ]:
# Using the bw_table, we will now go through each row/marker type and input the CTL and SSM2 bigwig files into the deeptools computeMatrix program
# This will create a matrix from both CTL and SSM2 bigwigs that can later be used to create heatmaps

for i in range(len(bw_table['mark'])):
    !computeMatrix reference-point -S {bw_table.loc[i, 'mark'] + "_CTL_log2.bw"} {bw_table.loc[i, 'mark'] + "_SSM2_log2.bw"} -R mm10_UCSC_knownGene.bed -bs 25 -b 5000 -a 5000  -out {bw_table.loc[i, 'mark'] + "_CTL_SSM2_log2_reference_point.matrix"} --missingDataAsZero --skipZeros --samplesLabel {bw_table.loc[i, 'mark'] + "_CTL"} {bw_table.loc[i, 'mark'] + "_SSM2"} -p 15

# Similar to the code above for running computeMatrix, we will use the bw_table to run deeptool's plotHeatmap
# This will make heatmap pairs of CTL and SSM2

for i in range(len(bw_table['mark'])):
    !plotHeatmap -m {bw_table.loc[i, 'mark'] + "_CTL_SSM2_log2_referencePoint.matrix"} --colorMap RdBu --heatmapHeight 25 --heatmapWidth 5 -out {bw_table.loc[i, 'mark'] + "_CTL_SSM2_log2_referencePoint.svg"} --sortUsing max --regionsLabel genes --legendLocation none

# Combined heatmaps around TSS, using absolute values, and curated gene lists

All the bw files and different histone modifications were implemented and processed together. 2 different heatmaps will ultimately be generated, with one utilizing 3 different bed files (cluster 1, cluster 2, and cluster 3), and the other only used a list of Hox genes.

In [27]:
#For the sake of easier name calling of bw files for combined heatmaps, we will rename them with more obvious names.
#To do this, we will run a for loop on the bw_table and use the os.rename() function.

for i in range(len(bw_table['mark'])):
    for n in range(len(bw_table.columns)-1):
        old_name = bw_table.loc[i, bw_table.columns[n+1]]
        new_name = bw_table.loc[i, bw_table.columns[0]] + "_" + bw_table.columns[n+1] + ".bw"
        print("Old name = " + old_name + " New name = " +  new_name)
        os.rename(old_name, new_name)

Old name = PX3242_CACTCA-GTAGTACT.markdup.q5.F3332.PET.bw New name = H3K27ac_CTL.bw
Old name = PX3242_CACGAT-CAGTACAG.markdup.q5.F3332.PET.bw New name = H3K27ac_SSM2.bw
Old name = PX3242_AAACAT-GTAGTACT.markdup.q5.F3332.PET.bw New name = H3K4me3_CTL.bw
Old name = PX3242_CTTGTA-CAGTACAG.markdup.q5.F3332.PET.bw New name = H3K4me3_SSM2.bw
Old name = PX3242_CCGCAA-GTAGTACT.markdup.q5.F3332.PET.bw New name = H3K36me2_CTL.bw
Old name = PX3242_GAGTGG-CAGTACAG.markdup.q5.F3332.PET.bw New name = H3K36me2_SSM2.bw
Old name = PX3242_GTGAAA-GTAGTACT.markdup.q5.F3332.PET.bw New name = Input_CTL.bw
Old name = PX3242_TTCTCC-GTAGTACT.markdup.q5.F3332.PET.bw New name = Input_SSM2.bw
Old name = PX3242_ATCCTA-GTAGTACT.markdup.q5.F3332.PET.bw New name = H3K36me3_CTL.bw
Old name = PX3242_ATACGG-CAGTACAG.markdup.q5.F3332.PET.bw New name = H3K36me3_SSM2.bw
Old name = PX3242_AAGGAC-GTAGTACT.markdup.q5.F3332.PET.bw New name = H3K9me3_CTL.bw
Old name = PX3242_AAGCGA-CAGTACAG.markdup.q5.F3332.PET.bw New name = H3

In [ ]:
#Compute a heatmap matrix that combines all 16 different bw files in a specific order and based around select regions using bed files.
#The following will utilize non-normalized bw files.
#Two separate heatmaps will be ultimately generated, one with 3 different clusters using gene lists, and one that uses Hox genes.
#Will generate matrices that calculate scores 1 kb upstream and downstream of the transciption start site (TSS).

!computeMatrix reference-point \
    -S H3K4me3_CTL.bw H3K4me3_SSM2.bw H3K27ac_CTL.bw H3K27ac_SSM2.bw H3K27me3_CTL.bw H3K27me3_SSM2.bw Ubiquityl_CTL.bw Ubiquityl_SSM2.bw H3K4me1_CTL.bw H3K4me1_SSM2.bw H3K9me3_CTL.bw H3K9me3_SSM2.bw H3K36me2_CTL.bw H3K36me2_SSM2.bw H3K36me3_CTL.bw H3K36me3_SSM2.bw \
    -R mm10_cluster1_genelist_NCBI.bed mm10_cluster2_genelist_NCBI.bed mm10_cluster3_genelist_NCBI.bed \
    -bs 25 -b 5000 -a 5000  -out CTL_SSM2_TSS_5kb_clusters.matrix --missingDataAsZero --skipZeros \
    --samplesLabel H3K4me3_CTL H3K4me3_SSM2 H3K27ac_CTL H3K27ac_SSM2 H3K27me3_CTL H3K27me3_SSM2 Ubiquityl_CTL Ubiquityl_SSM2 H3K4me1_CTL H3K4me1_SSM2 H3K9me3_CTL H3K9me3_SSM2 H3K36me2_CTL H3K36me2_SSM2 H3K36me3_CTL H3K36me3_SSM2 \
    -p 5 --verbose

!computeMatrix reference-point \
    -S H3K4me3_CTL.bw H3K4me3_SSM2.bw H3K27ac_CTL.bw H3K27ac_SSM2.bw H3K27me3_CTL.bw H3K27me3_SSM2.bw Ubiquityl_CTL.bw Ubiquityl_SSM2.bw H3K4me1_CTL.bw H3K4me1_SSM2.bw H3K9me3_CTL.bw H3K9me3_SSM2.bw H3K36me2_CTL.bw H3K36me2_SSM2.bw H3K36me3_CTL.bw H3K36me3_SSM2.bw \
    -R mm10_HoxGenes_genelist_NCBI.bed \
    -bs 25 -b 5000 -a 5000  -out CTL_SSM2_TSS_5kb_HoxGenes.matrix --missingDataAsZero --skipZeros \
    --samplesLabel H3K4me3_CTL H3K4me3_SSM2 H3K27ac_CTL H3K27ac_SSM2 H3K27me3_CTL H3K27me3_SSM2 Ubiquityl_CTL Ubiquityl_SSM2 H3K4me1_CTL H3K4me1_SSM2 H3K9me3_CTL H3K9me3_SSM2 H3K36me2_CTL H3K36me2_SSM2 H3K36me3_CTL H3K36me3_SSM2 \
    -p 5 --verbose

In [ ]:
#Generate heatmaps in svg format for both types.

!plotHeatmap -m CTL_SSM2_TSS_5kb_clusters.matrix --colorMap RdBu --heatmapHeight 25 --heatmapWidth 5 \
    -out CTL_SSM2_TSS_5kb_clusters.svg --sortUsing max --legendLocation none \
    --regionsLabel "Cluster 1" "Cluster 2" "Cluster 3" --whatToShow "heatmap and colorbar"

plotHeatmap -m CTL_SSM2_TSS_5kb_HoxGenes.matrix --colorMap RdBu --heatmapHeight 25 --heatmapWidth 5 \
    -out CTL_SSM2_TSS_5kb_HoxGenes.svg --sortUsing max --legendLocation none \
    --regionsLabel "Hox Genes" --whatToShow "heatmap and colorbar"

# Combined heatmaps around TSS, using log2 normailzed values, and curated gene lists

All the bw files and different histone modifications were implemented and processed together. 2 different heatmaps will ultimately be generated, with one utilizing 3 different bed files (cluster 1, cluster 2, and cluster 3), and the other only used a list of Hox genes.

In [38]:
#Compute a heatmap matrix that combines all 16 different bw files in a specific order and based around select regions using bed files.
#The following will utilize non-normalized bw files.
#Two separate heatmaps will be ultimately generated, one with 3 different clusters using gene lists, and one that uses Hox genes.
#Will generate matrices that calculate scores 1 kb upstream and downstream of the transciption start site (TSS).

!computeMatrix reference-point \
    -S H3K4me3_CTL_log2.bw H3K4me3_SSM2_log2.bw H3K27ac_CTL_log2.bw H3K27ac_SSM2_log2.bw H3K27me3_CTL_log2.bw H3K27me3_SSM2_log2.bw Ubiquityl_CTL_log2.bw Ubiquityl_SSM2_log2.bw H3K4me1_CTL_log2.bw H3K4me1_SSM2_log2.bw H3K9me3_CTL_log2.bw H3K9me3_SSM2_log2.bw H3K36me2_CTL_log2.bw H3K36me2_SSM2_log2.bw H3K36me3_CTL_log2.bw H3K36me3_SSM2_log2.bw \
    -R mm10_cluster1_genelist_NCBI_UCSC_filtered.bed mm10_cluster2_genelist_NCBI_UCSC_filtered.bed mm10_cluster3_genelist_NCBI_UCSC_filtered.bed -bs 25 -b 5000 -a 5000  \
    -out CTL_SSM2_TSS_log2_5kb_clusters_filtered.matrix --missingDataAsZero --skipZeros \
    --samplesLabel H3K4me3_CTL H3K4me3_SSM2 H3K27ac_CTL H3K27ac_SSM2 H3K27me3_CTL H3K27me3_SSM2 Ubiquityl_CTL Ubiquityl_SSM2 H3K4me1_CTL H3K4me1_SSM2 H3K9me3_CTL H3K9me3_SSM2 H3K36me2_CTL H3K36me2_SSM2 H3K36me3_CTL H3K36me3_SSM2 \
    -p 20 --verbose

!computeMatrix reference-point \
    -S H3K4me3_CTL_log2.bw H3K4me3_SSM2_log2.bw H3K27ac_CTL_log2.bw H3K27ac_SSM2_log2.bw H3K27me3_CTL_log2.bw H3K27me3_SSM2_log2.bw Ubiquityl_CTL_log2.bw Ubiquityl_SSM2_log2.bw H3K4me1_CTL_log2.bw H3K4me1_SSM2_log2.bw H3K9me3_CTL_log2.bw H3K9me3_SSM2_log2.bw H3K36me2_CTL_log2.bw H3K36me2_SSM2_log2.bw H3K36me3_CTL_log2.bw H3K36me3_SSM2_log2.bw \
    -R mm10_HoxGenes_genelist_NCBI_UCSC_filtered.bed -bs 25 -b 5000 -a 5000  \
    -out CTL_SSM2_TSS_log2_5kb_HoxGenes_filtered.matrix --missingDataAsZero --skipZeros \
    --samplesLabel H3K4me3_CTL H3K4me3_SSM2 H3K27ac_CTL H3K27ac_SSM2 H3K27me3_CTL H3K27me3_SSM2 Ubiquityl_CTL Ubiquityl_SSM2 H3K4me1_CTL H3K4me1_SSM2 H3K9me3_CTL H3K9me3_SSM2 H3K36me2_CTL H3K36me2_SSM2 H3K36me3_CTL H3K36me3_SSM2 \
    -p 20 --verbose


The following chromosome names did not match between the bigwig files
chromosome	length
  chrJH584294.1	1073741823
  chrGL456350.1	1073741823
  chrJH584293.1	1073741823
  chrJH584303.1	1073741823
genome partition size for multiprocessing: 100000
using 10 processors for 8 number of tasks
Processing chr2:74600000-74700000
Processing chr15:102900000-103000000
Processing chr11:96200000-96300000
Processing chr11:96300000-96400000
Processing chr6:52100000-52200000
Processing chr15:103000000-103100000
Processing chr6:52200000-52300000
Processing chr2:74700000-74800000


In [42]:
!plotHeatmap -m CTL_SSM2_TSS_log2_5kb_clusters_filtered_2.matrix --colorMap RdBu --heatmapHeight 25 --heatmapWidth 5 \
    -out CTL_SSM2_TSS_5kb_clusters_filtered.svg --sortUsing max --legendLocation none \
    --regionsLabel "Cluster 1" "Cluster 2" "Cluster 3" --whatToShow "heatmap and colorbar" --outFileSortedRegions "CTL_SSM2_TSS_5kb_clusters_filtered.bed"

!plotHeatmap -m CTL_SSM2_TSS_log2_5kb_HoxGenes_filtered.matrix --colorMap RdBu --heatmapHeight 10 --heatmapWidth 5 \
    -out CTL_SSM2_TSS_5kb_HoxGenes_filtered.svg --sortUsing max --legendLocation none \
    --regionsLabel "Hox Genes" --whatToShow "heatmap and colorbar" --outFileSortedRegions "CTL_SSM2_TSS_5kb_HoxGenes_filtered.bed"